# Script for PreProcessing for Water Consumption Prediction

### Import Dependencies

In [ ]:
import os
import yaml
from numpy import array

import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import LSTM, Dropout,Dense,RepeatVector,TimeDistributed,Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam as adam
from sklearn.model_selection import train_test_split

import datetime

## Load the Data

In [ ]:
df = pd.read_csv("sample.txt")

In [ ]:
df.head()

,id,starttime,stoptime,water_consumption
0,B,2020-12-21 17:49:09.000,2020-12-21 17:50:05.000,3.8
1,B,2020-12-22 20:08:54.000,2020-12-22 20:10:01.000,6.2
2,B,2020-09-09 17:54:47.000,2020-09-09 17:56:02.000,3.7
3,B,2020-12-05 06:49:33.000,2020-12-05 06:50:55.000,7.7
4,B,2020-12-30 08:54:29.000,2020-12-30 08:55:57.000,8.1


In [ ]:
df.shape

(65505, 4)

### Resample the Data Household Hour Wise

### For Household B

In [ ]:
# Select Household B
dfB = df.loc[df['id'] == "B"]

# Sort the time time and date wise
dfB = dfB.sort_values(by="starttime")

# Delete Start time and Id as it is not required for sampling as Stop time is sufficient
del dfB["starttime"]
del dfB["id"]
dfB.columns = ["time", "water_consumption_B"]

dfB['time']= pd.to_datetime(dfB['time'])
dfB = dfB.set_index("time")

# Resample on hourly basis and sum all values within one hour
dfB = dfB.resample("H").sum()
dfB.head()

,water_consumption_B
time,
2020-07-01 04:00:00,40.8
2020-07-01 05:00:00,36.6
2020-07-01 06:00:00,0.0
2020-07-01 07:00:00,0.0
2020-07-01 08:00:00,0.0


### Similarly For Household A, C, D, E

In [ ]:
dfA = df.loc[df['id'] == "A"]
dfA = dfA.sort_values(by="starttime")
del dfA["starttime"]
del dfA["id"]
dfA.columns = ["time", "water_consumption_A"]
dfA['time']= pd.to_datetime(dfA['time'])
dfA = dfA.set_index("time")
dfA = dfA.resample("H").sum()
dfA.head()

,water_consumption_A
time,
2020-07-01 05:00:00,8.7
2020-07-01 06:00:00,4.8
2020-07-01 07:00:00,7.6
2020-07-01 08:00:00,48.4
2020-07-01 09:00:00,6.8


In [ ]:
dfC = df.loc[df['id'] == "C"]
dfC = dfC.sort_values(by="starttime")
del dfC["starttime"]
del dfC["id"]
dfC.columns = ["time", "water_consumption_C"]
dfC['time']= pd.to_datetime(dfC['time'])
dfC = dfC.set_index("time")
dfC = dfC.resample("H").sum()
dfC.head()

,water_consumption_C
time,
2020-07-24 06:00:00,17.6
2020-07-24 07:00:00,105.6
2020-07-24 08:00:00,8.5
2020-07-24 09:00:00,6.8
2020-07-24 10:00:00,7.2


In [ ]:
dfD = df.loc[df['id'] == "D"]
dfD = dfD.sort_values(by="starttime")
del dfD["starttime"]
del dfD["id"]
dfD.columns = ["time", "water_consumption_D"]
dfD['time']= pd.to_datetime(dfD['time'])
dfD = dfD.set_index("time")
dfD = dfD.resample("H").sum()
dfD.head()

,water_consumption_D
time,
2020-07-01 04:00:00,8.0
2020-07-01 05:00:00,5.1
2020-07-01 06:00:00,0.0
2020-07-01 07:00:00,18.6
2020-07-01 08:00:00,20.1


In [ ]:
dfE = df.loc[df['id'] == "E"]
dfE = dfE.sort_values(by="starttime")
del dfE["starttime"]
del dfE["id"]
dfE.columns = ["time", "water_consumption_E"]
dfE['time']= pd.to_datetime(dfE['time'])
dfE = dfE.set_index("time")
dfE = dfE.resample("H").sum()
dfE.head()

,water_consumption_E
time,
2020-07-01 01:00:00,6.4
2020-07-01 02:00:00,0.0
2020-07-01 03:00:00,0.0
2020-07-01 04:00:00,6.7
2020-07-01 05:00:00,0.8


## Make a Final Pre-Processed Dataset

In [ ]:
df_processed_1 = pd.merge(dfB, dfA, left_index = True, right_index = True, how = 'left')
df_processed_2 = pd.merge(dfD, dfC, left_index = True, right_index = True, how = 'left')
df_processed_3 = pd.merge(df_processed_1, df_processed_2, left_index = True, right_index = True, how = 'left')
df_processed = pd.merge(df_processed_3, dfE, left_index = True, right_index = True, how = 'left')
df_processed.head()

,water_consumption_B,water_consumption_A,water_consumption_D,water_consumption_C,water_consumption_E
time,,,,,
2020-07-01 04:00:00,40.8,NaN,8.0,NaN,6.7
2020-07-01 05:00:00,36.6,8.7,5.1,NaN,0.8
2020-07-01 06:00:00,0.0,4.8,0.0,NaN,0.0
2020-07-01 07:00:00,0.0,7.6,18.6,NaN,0.0
2020-07-01 08:00:00,0.0,48.4,20.1,NaN,0.0


In [ ]:
# Fill NaN values with 0
df_processed = df_processed.fillna(0)

### Save Preprocessed Data

In [ ]:
# Save Preprocessed Data
df_processed.to_csv("data_resampled_hour.csv")